In [3]:
# cirqライブラリをインストールする。
import cirq
import math 

# 真にランダムバイトを生成するメイン関数
def main():
    # QPU関数呼び出し(qcインスタンスを生成する。)
    qc = QPU()
    # 1つのキュビットを割り当てる。
    qc.reset(1)
    
    # root-of-not回路(1回目)
    qc.had();
    qc.phase(-90);
    qc.had();
    # root-of-not回路(2回目)
    qc.had();
    qc.phase(-90);
    qc.had();
    
    # この時点で入力時と逆となる。
    # 以下、root-of-not命令2回実行 = not命令
    qc.rootnot()
    qc.rootnot()
    
    # キュビットを観測
    qc.read() 
    # 円を描く
    qc.draw() 
    result = qc.run() 
    # 出力する。
    print(result)
    
    

######################################################################
## Criqで用意されているインターフェース群
class QPU:
    def __init__(self):
        self.circuit = cirq.Circuit()
        self.simulator = cirq.Simulator()
        self.qubits = None

    def reset(self, num_qubits):
        self.qubits = [cirq.GridQubit(i, 0) for i in range(num_qubits)]

    def mask_to_list(self, mask):
        return [q for i,q in enumerate(self.qubits) if (1 << i) & mask]

    def had(self, target_mask=~0):
        target = self.mask_to_list(target_mask)
        self.circuit.append(cirq.H.on_each(*target))

    def phase(self, theta_degrees, target_mask=~0):
        target = self.mask_to_list(target_mask)
        theta_radians = theta_degrees * math.pi / 180.0
        self.circuit.append(cirq.Rz(theta_radians).on_each(*target))

    def rootnot(self, target_mask=~0):
        sqrt_x = cirq.X**0.5
        target = self.mask_to_list(target_mask)
        self.circuit.append(sqrt_x.on_each(*target))

    def read(self, target_mask=~0, key=None):
        if key is None:
            key = 'result'
        target = self.mask_to_list(target_mask)
        self.circuit.append(cirq.measure(*target, key=key))

    def draw(self):
        print('Circuit:\n{}'.format(self.circuit))

    def run(self):
        return self.simulator.simulate(self.circuit)


    
# インポートされた際にプログラムが動かないようにする。
if __name__ == '__main__':
    main()

TypeError: __init__() takes 1 positional argument but 2 were given